# The Ultimate Algo Trader (Neurobot 1.0)

## Introduction:
This jupyter notebook contains code for developing and testing an algorithmic trader based on Python and Machine learning strategy.

## Steps:
1. **Data preperation**: Access historical market data via Alpaca API and preprocess it for analysis.
2. **Strategy**: Develop algorithmic trading strategies and instantiate LSTM model.
3. **Backtesting-Risk/Reward**: Backtest the strategy using historical data and calculate both the risk and the reward metrics
4. **live Trading(Optional)**: Implement the strategy for paper trading on Alpaca via API.

#### Notes: 
- This notebook is for educational and experimental purposes only.

### Imports and Dependencies

In [2]:
import os
import numpy as np
import random
import datetime
import seaborn as sns
import pandas as pd
import yfinance as yf
import hvplot.pandas
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
from scipy.interpolate import interp1d
from alpaca_trade_api.rest import REST, TimeFrame
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import mean_squared_error
import pandas_ta as ta
from pandas.tseries.offsets import DateOffset
from sklearn.preprocessing import StandardScaler
from finta import TA
import holoviews as hv
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


# Initialize python files and import functions
import stock_data as data
import algo_strategy as strategy
import backtesting as test
import feature_list as feature

import warnings
warnings.filterwarnings('ignore')


In [3]:
load_dotenv()

True

In [4]:
# Set API credentials
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
base_url = 'https://paper-api.alpaca.markets'

# ticker list based on monthly highest dollar volume rank(liquidity) 
tickers_list = data.get_top_tickers(sp500_url = os.getenv("SP500_URL"))
tickers = [ticker[1] for ticker in tickers_list]

# Call the function to fetch stock data
stock_data = data.fetch_stock_data(tickers, alpaca_api_key, alpaca_secret_key, base_url)
stock_df = stock_data.dropna()

[*********************100%%**********************]  503 of 503 completed


In [5]:
print(tickers)

['ZTS', 'CHTR', 'ITW', 'CAT', 'CBRE', 'CCI', 'CDNS', 'CDW', 'CE', 'CHD', 'CI', 'BKNG', 'CLX', 'IRM', 'CMCSA', 'COF', 'COR', 'IPG', 'IP', 'CRL', 'JBL', 'BIO', 'HII', 'AME', 'KR', 'AEE', 'AFL', 'AIZ', 'AKAM', 'ALGN', 'AMAT', 'AMCR', 'AMGN', 'BDX', 'AMT', 'AMZN', 'APD', 'AZO', 'CMS', 'BALL', 'BAX', 'K', 'CTLT', 'CTVA', 'CZR', 'HPQ', 'FANG', 'HUBB', 'HSY', 'FI', 'FOXA', 'FRT', 'FSLR', 'FTNT', 'GEHC', 'DAY', 'GEN', 'HPE', 'GNRC', 'GPC', 'GPN', 'HIG', 'HAS', 'HES', 'F', 'EXPE', 'HWM', 'EW', 'DE', 'DFS', 'INCY', 'DIS', 'DLR', 'DOC', 'DPZ']


In [6]:
stock_df

,symbol,open,high,low,close,volume
timestamp,,,,,,
2016-03-01 05:00:00+00:00,AEE,47.190,47.5400,46.53,46.93,1982704
2016-03-02 05:00:00+00:00,AEE,46.650,46.9800,45.56,46.96,2166441
2016-03-03 05:00:00+00:00,AEE,46.955,47.0000,46.23,47.00,1613514
2016-03-04 05:00:00+00:00,AEE,46.780,47.5989,46.52,47.35,1931980
2016-03-07 05:00:00+00:00,AEE,46.830,47.4400,46.70,47.13,1570908
...,...,...,...,...,...,...
2024-02-26 05:00:00+00:00,ZTS,196.900,197.9900,195.46,197.04,2098518
2024-02-27 05:00:00+00:00,ZTS,197.650,199.2500,194.04,199.00,1984613
2024-02-28 05:00:00+00:00,ZTS,199.300,200.5000,197.10,199.94,2003263


#### Feauture Engineering - Time series Analysis

In [7]:
daily_returns = feature.calculate_pct_change(stock_df)

In [8]:
cumulative_returns = feature.cumulative_returns(stock_df)

In [9]:
sma = feature.simple_moving_averge(stock_df)

In [10]:
sma_ema_long_short = feature.sma_ema_long_short(stock_df)

In [11]:
standard_deviation = feature.standard_deviation(stock_df)

In [12]:
Upper_lower_bollinger = feature.calculate_bollinger_bands(stock_df)

In [13]:
finta = feature.finta_technical_indicators(stock_df)

### Data Processing and Cleaning

In [14]:
# Drop NAN values
stock_df.dropna(inplace=True)
display(stock_df.head())
display(stock_df.tail())


,symbol,open,high,low,close,volume,Daily Returns,Cumulative Returns,SMA20,SMA50,SMA100,STD,Upper Band,Lower Band,TA Short,TA Long
timestamp,,,,,,,,,,,,,,,,
2016-07-21 04:00:00+00:00,AEE,52.04,52.28,51.675,52.24,2283504,0.000575,1.113147,52.6685,50.5876,49.5094,0.648068,53.964636,51.372364,52.866000,50.5876
2016-07-22 04:00:00+00:00,AEE,52.24,52.97,52.240,52.87,1872869,0.012060,1.126571,52.7370,50.6820,49.5688,0.587646,53.912291,51.561709,52.818667,50.6820
2016-07-25 04:00:00+00:00,AEE,52.83,52.94,52.420,52.70,840357,-0.003215,1.122949,52.7825,50.7736,49.6262,0.544077,53.870654,51.694346,52.748667,50.7736
2016-07-26 04:00:00+00:00,AEE,52.85,52.85,51.950,52.22,1280711,-0.009108,1.112721,52.7780,50.8566,49.6784,0.548544,53.875089,51.680911,52.646000,50.8566
2016-07-27 04:00:00+00:00,AEE,52.06,52.25,51.260,51.77,1600067,-0.008617,1.103132,52.7525,50.9340,49.7226,0.583644,53.919789,51.585211,52.512667,50.9340


,symbol,open,high,low,close,volume,Daily Returns,Cumulative Returns,SMA20,SMA50,SMA100,STD,Upper Band,Lower Band,TA Short,TA Long
timestamp,,,,,,,,,,,,,,,,
2024-02-26 05:00:00+00:00,ZTS,196.90,197.9900,195.46,197.04,2098518,-0.000862,4.198594,191.7395,193.3292,182.8673,4.676759,201.093019,182.385981,192.307333,193.3292
2024-02-27 05:00:00+00:00,ZTS,197.65,199.2500,194.04,199.00,1984613,0.009947,4.240358,192.0515,193.3610,183.1540,4.948663,201.948826,182.154174,192.896000,193.3610
2024-02-28 05:00:00+00:00,ZTS,199.30,200.5000,197.10,199.94,2003263,0.004724,4.260388,192.4770,193.3580,183.4435,5.249146,202.975293,181.978707,193.149333,193.3580
2024-02-29 05:00:00+00:00,ZTS,200.50,200.5341,198.19,198.33,2849159,-0.008052,4.226081,193.0030,193.3988,183.7068,5.283840,203.570681,182.435319,193.232000,193.3988
2024-03-01 05:00:00+00:00,ZTS,196.00,196.2300,188.49,190.82,3724389,-0.037866,4.066056,193.0870,193.2808,183.8592,5.232299,203.551597,182.622403,192.903333,193.2808


### Algorithim Buy/Sell  Best signal selection

In [15]:
simple_momentum = strategy.simple_momentum(stock_df)
dmac= strategy.dmac_strategy(stock_df)
finta_strategy = strategy.finta_strategy(stock_df)
bollinger_bands = strategy.bollinger_bands_strategy(stock_df)
pairs_trading = strategy.pairs_trading_signals(stock_df, entry_threshold=1.0, exit_threshold=0.5)

In [16]:
signal_one = stock_df['Signal_SM']
signal_two = stock_df['Signal_DMAC']
signal_three = stock_df['Signal_FINTA']
signal_four = stock_df['Signal_BB']
signal_five = stock_df['Signal_PTS']

In [17]:
# Create a list of lists containing signals for each asset
signals = [signal_one, signal_two, signal_three, signal_four, signal_five]

In [18]:
def majority_vote(signals):
    majority_signals = []

    # Iterate over each asset's signals
    for asset_signals in zip(*signals):
        buy_count = sum(1 for signal in asset_signals if signal == 1)
        sell_count = sum(1 for signal in asset_signals if signal == -1)
        
        # Determine the majority vote for the current asset
        if buy_count > sell_count:
            majority_signal = 1  # Buy signal
        elif sell_count > buy_count:
            majority_signal = -1  # Sell signal
        else:
            majority_signal = 0  # Hold signal
        
        # Append the majority signal for the current asset to the list
        majority_signals.append(majority_signal)

    return majority_signals

# Call the majority_vote function with the list of signals
majority_signals = majority_vote(signals)

# Add the majority signals as a new column to the existing stock_df DataFrame
stock_df['Signal'] = majority_signals

# Add entry and exit columns for plotting entry and exit points
stock_df['entry'] = stock_df['majority_signal'].apply(lambda x: 1 if x == 1 else 0)
stock_df['exit'] = stock_df['majority_signal'].apply(lambda x: -1 if x == -1 else 0)

# Print the updated stock_df DataFrame
stock_df


,symbol,open,high,low,close,volume,Daily Returns,Cumulative Returns,SMA20,SMA50,...,TA Long,Signal_SM,Signal_DMAC,Signal_FINTA,Signal_BB,Spread,Signal_PTS,majority_signal,entry,exit
timestamp,,,,,,,,,,,,,,,,,,,,,
2016-07-21 04:00:00+00:00,AEE,52.04,52.2800,51.675,52.24,2283504,0.000575,1.113147,52.6685,50.5876,...,50.5876,1.0,0.0,0.0,0.0,0.20,0,1,1,0
2016-07-22 04:00:00+00:00,AEE,52.24,52.9700,52.240,52.87,1872869,0.012060,1.126571,52.7370,50.6820,...,50.6820,1.0,0.0,0.0,0.0,0.63,0,1,1,0
2016-07-25 04:00:00+00:00,AEE,52.83,52.9400,52.420,52.70,840357,-0.003215,1.122949,52.7825,50.7736,...,50.7736,-1.0,0.0,0.0,0.0,-0.13,0,-1,0,-1
2016-07-26 04:00:00+00:00,AEE,52.85,52.8500,51.950,52.22,1280711,-0.009108,1.112721,52.7780,50.8566,...,50.8566,-1.0,0.0,0.0,0.0,-0.63,0,-1,0,-1
2016-07-27 04:00:00+00:00,AEE,52.06,52.2500,51.260,51.77,1600067,-0.008617,1.103132,52.7525,50.9340,...,50.9340,-1.0,0.0,0.0,0.0,-0.29,0,-1,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-26 05:00:00+00:00,ZTS,196.90,197.9900,195.460,197.04,2098518,-0.000862,4.198594,191.7395,193.3292,...,193.3292,-1.0,1.0,0.0,0.0,0.14,0,0,0,0
2024-02-27 05:00:00+00:00,ZTS,197.65,199.2500,194.040,199.00,1984613,0.009947,4.240358,192.0515,193.3610,...,193.3610,1.0,1.0,0.0,0.0,1.35,1,1,1,0
2024-02-28 05:00:00+00:00,ZTS,199.30,200.5000,197.100,199.94,2003263,0.004724,4.260388,192.4770,193.3580,...,193.3580,1.0,1.0,0.0,0.0,0.64,0,1,1,0


### Train Test Split (DateOffset)

In [38]:
from sklearn.model_selection import train_test_split

# Define your features (X) and target variable (y)
X = stock_df.drop(columns=["majority_signal","symbol"])  # Features
y = stock_df["majority_signal"]  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Now you have X_train, X_test, y_train, and y_test ready for training your model


In [23]:
# Setting x and y labels
#stock_df.reset_index(inplace=True)
#stock_df.set_index(['timestamp', 'symbol'], inplace=True)

# Create X by copying the dataframe and excluding unnecessary columns
X = stock_df.drop(columns=["majority_signal"]).shift(-1).dropna().copy()

# Create y containing only the "Signal" column
y = stock_df["majority_signal"].copy()

training_begin = X.index.min()

training_end = X.index.min() + DateOffset(months=68)

X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end+DateOffset(months=27):]
y_test = y.loc[training_end+DateOffset(months=27):]

KeyError: "Cannot get left slice bound for non-unique label: Timestamp('2016-03-01 05:00:00+0000', tz='UTC')"

### Standardize The Data

In [39]:
# Create a StandardScaler instance
scaler = StandardScaler()
 
# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)
 
# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### PCA - dimension reduction

### Machine Learning Best Model Selection

##### Data reshape

In [40]:
timesteps = 5

# Adjust the range to keep the same number of samples for X_train_reshaped
X_train_reshaped = np.array([X_train_scaled[i:i+timesteps] for i in range(len(X_train_scaled) - timesteps + 1)])
y_train_reshaped = y_train[:len(X_train_reshaped)]  # Adjust y_train accordingly

print("Shape of X_train_reshaped:", X_train_reshaped.shape)
print("Shape of y_train_reshaped:", y_train_reshaped.shape)

# Generate the X_test and y_test DataFrames with appropriate indexing
X_test_reshaped = np.array([X_test_scaled[i:i+timesteps] for i in range(len(X_test_scaled) - timesteps + 1)])
y_test_reshaped = y_test[:len(X_test_reshaped)]  # Adjust y_test accordingly

print("Shape of X_test_reshaped:", X_test_reshaped.shape)
print("Shape of y_test_reshaped:", y_test_reshaped.shape)

Shape of X_train_reshaped: (102264, 5, 23)
Shape of y_train_reshaped: (102264,)
Shape of X_test_reshaped: (43826, 5, 23)
Shape of y_test_reshaped: (43826,)


In [41]:
#LSTM MODEL

model = Sequential([
    LSTM(units=50, return_sequences=True, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])),
    LSTM(units=50, return_sequences=False),
    Dense(units=1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_reshaped, y_train_reshaped, epochs=100, batch_size=32, validation_data=(X_test_reshaped, y_test_reshaped))

Epoch 1/100
3196/3196 [==============================] - 38s 10ms/step - loss: -14.1078 - accuracy: 0.8295 - val_loss: -26.3900 - val_accuracy: 0.8383
Epoch 2/100
3196/3196 [==============================] - 32s 10ms/step - loss: -38.8274 - accuracy: 0.8353 - val_loss: -50.7065 - val_accuracy: 0.8378
Epoch 3/100
3196/3196 [==============================] - 32s 10ms/step - loss: -63.6188 - accuracy: 0.8364 - val_loss: -75.1339 - val_accuracy: 0.8389
Epoch 4/100
3196/3196 [==============================] - 36s 11ms/step - loss: -88.0381 - accuracy: 0.8366 - val_loss: -99.4355 - val_accuracy: 0.8387
Epoch 5/100
3196/3196 [==============================] - 36s 11ms/step - loss: -112.9593 - accuracy: 0.8368 - val_loss: -123.8731 - val_accuracy: 0.8387
Epoch 6/100
3196/3196 [==============================] - 32s 10ms/step - loss: -137.6137 - accuracy: 0.8370 - val_loss: -148.2581 - val_accuracy: 0.8389
Epoch 7/100
3196/3196 [==============================] - 32s 10ms/step - loss: -162.1830 -

#### LSTM Prediction

In [45]:
prediction_df = model.predict(y_train)

ValueError: in user code:

    File "C:\Users\user\anaconda3\envs\dev\lib\site-packages\keras\src\engine\training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\user\anaconda3\envs\dev\lib\site-packages\keras\src\engine\training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\user\anaconda3\envs\dev\lib\site-packages\keras\src\engine\training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\user\anaconda3\envs\dev\lib\site-packages\keras\src\engine\training.py", line 2381, in predict_step
        return self(x, training=False)
    File "C:\Users\user\anaconda3\envs\dev\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\user\anaconda3\envs\dev\lib\site-packages\keras\src\engine\input_spec.py", line 235, in a

In [42]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_reshaped, y_test_reshaped)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# can use stock prices for label, and change metrics and loss to mse

1370/1370 [==============================] - 6s 4ms/step - loss: -2438.7407 - accuracy: 0.8389
Test Loss: -2438.74072265625
Test Accuracy: 0.8389084339141846


### Backtesting - Trained Model against Test Model 

In [ ]:
subset_df = backtest.subset_around_crossovers(stock_df, all_cross_points)


#### Portfolio-Level Risk/Reward Evaluation Metrics

In [ ]:
portfolio_metric = test.portfolio_metrics(stock_df)

In [ ]:
# Calculate trade evaluation inplace
trade_evaluation = test.trade_evaluation(stock_df)

In [ ]:
stock_df

### Fundamental Analysis

In [ ]:
# Plot close price, entry, and exit points
stock_df.loc["2019-02-09":"2020-02-17"]
plt.figure(figsize=(10, 6))
plt.plot(stock_df.index, stock_df['close'], label='Close Price', color='b')
plt.scatter(stock_df.index[stock_df['entry'] == 1], stock_df['close'][stock_df['entry'] == 1], marker='^', color='green', label='Entry Point')
plt.scatter(stock_df.index[stock_df['exit'] == -1], stock_df['close'][stock_df['exit'] == -1], marker='v', color='red', label='Exit Point')  # Corrected to use -1 for exit
plt.title('Entry and Exit Points')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.legend()
plt.show()



In [ ]:
stock_df['Cumulative Returns'].plot()

In [ ]:
# Display the report
print(testing_report)

# Visually Compare the Actual and Predicted Returns
# Create a predictions DataFrame
predictions_df = pd.DataFrame(index=X_test.index)

predictions_df["predicted_signal"] = testing_signal_predictions

predictions_df["actual_returns"] = trading_df["actual_returns"]

predictions_df["trading_algorithm_returns"] = (
    predictions_df["actual_returns"] * predictions_df["predicted_signal"]
)

# Review the DataFrame
predictions_df.head()